### Required packages

In [2]:
import glob
import multiprocessing
import os
import shutil

import cv2
import keras
import mlflow
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import tqdm
from num2words import num2words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    LSTM,
    Concatenate,
    Conv1D,
    Dense,
    Embedding,
    GlobalMaxPooling1D,
    Input,
    MaxPooling1D,
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical


In [3]:
# number of patients from meta data
meta_data = pd.read_csv("./Dataset/HAM10000_metadata.csv")
print(f"Total Number of Patients: {meta_data.axes[0].size}")


Total Number of Patients: 10015


In [5]:
# total number of patients from images samples
p1 = len(os.listdir("./Dataset/HAM10000_images_part_1"))
p2 = len(os.listdir("./Dataset/HAM10000_images_part_2"))
print(f"Total Number of Patients: {p1+p2}")

Total Number of Patients: 10015


In [6]:
p1 = os.listdir("./Dataset/HAM10000_images_part_1")
p2 = os.listdir("./Dataset/HAM10000_images_part_2")
p1.extend(p2)
all_files_path = p1

In [13]:
# classifying each samples correctly from the meta data
p1_path = "./Dataset/HAM10000_images_part_1"
p2_path = "./Dataset/HAM10000_images_part_2"
onlyfiles = [
    f
    for f in all_files_path
    if os.path.isfile(os.path.join(p1_path, f))
    or os.path.isfile(os.path.join(p2_path, f))
]
# checking files existance
print(f"Total Number of existed sample of images: {len(onlyfiles)}")

Total Number of existed sample of images: 10015


In [6]:
# preparing dataset in a proper format for keras data loaders
# shutil.rmtree("HAM_10000_Dataset") # remove the folder for dataset if it already exists
os.makedirs("HAM_10000_Dataset", exist_ok=True)
# iterat on each sample and correctly classify it according its id in meta data
for dx in tqdm.tqdm(meta_data["dx"].unique(), desc="In Pregress", ascii=True):
    print(f"{dx} class files in progress")
    filterd_data = meta_data[meta_data["dx"] == dx]
    if dx not in os.listdir("HAM_10000_Dataset"):
        os.makedirs(f"HAM_10000_Dataset/{dx}")
        [
            shutil.copy(os.path.join(p1_path, f), f"HAM_10000_Dataset/{dx}/")
            for f in filterd_data["image_id"] + ".jpg"
            if os.path.isfile(os.path.join(p1_path, f))
        ]
        [
            shutil.copy(os.path.join(p2_path, f), f"HAM_10000_Dataset/{dx}/")
            for f in filterd_data["image_id"] + ".jpg"
            if os.path.isfile(os.path.join(p2_path, f))
        ]
    else:
        [
            shutil.copy(os.path.join(p1_path, f), f"HAM_10000_Dataset/{dx}/")
            for f in filterd_data["image_id"] + ".jpg"
            if os.path.isfile(os.path.join(p1_path, f))
        ]
        [
            shutil.copy(os.path.join(p2_path, f), f"HAM_10000_Dataset/{dx}/")
            for f in filterd_data["image_id"] + ".jpg"
            if os.path.isfile(os.path.join(p2_path, f))
        ]
print("Successfull Created Dataset for Data loaders")
print(
    f"Number of samples in total after creating dataset: {len(glob.glob('/kaggle/working/HAM_10000_Dataset/*/*'))}"
)

In Pregress:   0%|          | 0/7 [00:00<?, ?it/s]

bkl class files in progress


In Pregress:  14%|#4        | 1/7 [00:00<00:02,  2.33it/s]

nv class files in progress


In Pregress:  29%|##8       | 2/7 [00:05<00:17,  3.41s/it]

...

bcc class files in progress


In Pregress:  86%|########5 | 6/7 [00:14<00:02,  2.48s/it]

akiec class files in progress


In Pregress: 100%|##########| 7/7 [00:16<00:00,  2.33s/it]

Successfull Created Dataset for Data loaders
Number of samples in total after creating dataset: 0


### Data Loader

In [4]:
# here we will use data loaders to load and make it feedable for the model
# Define the directory path where the images are located
train_ds = tf.keras.utils.image_dataset_from_directory(
    "./HAM_10000_Dataset",
    validation_split=0.05,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "./HAM_10000_Dataset",
    validation_split=0.05,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32,
)

Found 10015 files belonging to 7 classes.
Using 9515 files for training.
Found 10015 files belonging to 7 classes.
Using 500 files for validation.


## Transfer Learning

In [37]:
pretrained_models = {
    "VGG16": keras.applications.VGG16,
    "ConvNeXtBase": keras.applications.ConvNeXtBase,
    "ConvNeXtLarge": keras.applications.ConvNeXtLarge,
    "ConvNeXtSmall": keras.applications.ConvNeXtSmall,
    "ConvNeXtTiny": keras.applications.ConvNeXtTiny,
    "DenseNet121": keras.applications.DenseNet121,
    "DenseNet169": keras.applications.DenseNet169,
    "DenseNet201": keras.applications.DenseNet201,
    "EfficientNetB0": keras.applications.EfficientNetB0,
    "EfficientNetB1": keras.applications.EfficientNetB1,
    "EfficientNetB2": keras.applications.EfficientNetB2,
    "EfficientNetB3": keras.applications.EfficientNetB3,
    "EfficientNetB4": keras.applications.EfficientNetB4,
    "EfficientNetB5": keras.applications.EfficientNetB5,
    "EfficientNetB6": keras.applications.EfficientNetB6,
    "EfficientNetB7": keras.applications.EfficientNetB7,
    "EfficientNetV2B0": keras.applications.EfficientNetV2B0,
    "EfficientNetV2B1": keras.applications.EfficientNetV2B1,
    "EfficientNetV2B2": keras.applications.EfficientNetV2B2,
    "EfficientNetV2B3": keras.applications.EfficientNetV2B3,
    "EfficientNetV2L": keras.applications.EfficientNetV2L,
    "EfficientNetV2M": keras.applications.EfficientNetV2M,
    "EfficientNetV2S": keras.applications.EfficientNetV2S,
    "InceptionResNetV2": keras.applications.InceptionResNetV2,
    "InceptionV3": keras.applications.InceptionV3,
    "MobileNet": keras.applications.MobileNet,
    "MobileNetV2": keras.applications.MobileNetV2,
    "MobileNetV3Large": keras.applications.MobileNetV3Large,
    "MobileNetV3Small": keras.applications.MobileNetV3Small,
    "NASNetLarge": keras.applications.NASNetLarge,
    "NASNetMobile": keras.applications.NASNetMobile,
    "ResNet101": keras.applications.ResNet101,
    "ResNet101V2": keras.applications.ResNet101V2,
    "ResNet152": keras.applications.ResNet152,
    "ResNet152V2": keras.applications.ResNet152V2,
    "ResNet50": keras.applications.ResNet50,
    "ResNet50V2": keras.applications.ResNet50V2,
    "ResNet152V2": keras.applications.ResNet152V2,
    "VGG19": keras.applications.VGG19,
    "Xception": keras.applications.Xception,
}

In [28]:
print(f"Total Number of Pretrained Models: {len(pretrained_models.keys())}")

Total Number of Pretrained Models: 30


In [29]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Image Data Experiments")

for key in pretrained_models.keys():
    try:
        print("=" * 40, f"{key} Model Training Started", "=" * 40)
        model = pretrained_models[key](include_top=False, input_shape=(224, 224, 3))
        for layer in model.layers:
            layer.trainable = False
        flat1 = keras.layers.Flatten()(model.layers[-1].output)
        class1 = keras.layers.Dense(512, activation="relu")(flat1)

        output = keras.layers.Dense(7, activation="softmax")(class1)
        # # define new model
        model = keras.Model(inputs=model.inputs, outputs=output)
        # model compilation
        model.compile(
            optimizer=keras.optimizers.Adam(),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"],
        )

        with mlflow.start_run(run_name=key, nested=True) as run:
            # log model parametes
            mlflow.log_param("Optimizer", "Adam")
            mlflow.log_param("Loss", "Sparse_Categorical_Crossentropy")
            mlflow.log_param("Batch_size", 32)
            mlflow.log_param("Epochs", 50)

            # fit the model
            history = model.fit(train_ds, epochs=50, validation_data=val_ds)

            # Log metrics
            for epoch in range(50):
                mlflow.log_metric(
                    "train_accuracy", history.history["accuracy"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "val_accuracy", history.history["val_accuracy"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "train_loss", history.history["loss"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "val_loss", history.history["val_loss"][epoch], step=epoch
                )

            # log the model
            # mlflow.keras.log_model(model,f"Keras-model-{key}")
    except Exception as e:
        print(e)

======================================== EfficientNetB1 Model Training Started ========================================
27018416/27018416 [==============================] - 3s 0us/step
Epoch 1/50
298/298 [==============================] - 30s 84ms/step - loss: 1.8487 - accuracy: 0.7066 - val_loss: 0.6788 - val_accuracy: 0.7540
Epoch 2/50
298/298 [==============================] - 25s 81ms/step - loss: 0.5525 - accuracy: 0.8003 - val_loss: 0.6318 - val_accuracy: 0.7780
Epoch 3/50
298/298 [==============================] - 24s 80ms/step - loss: 0.4330 - accuracy: 0.8444 - val_loss: 0.6844 - val_accuracy: 0.7700
Epoch 4/50
298/298 [==============================] - 25s 81ms/step - loss: 0.3422 - accuracy: 0.8785 - val_loss: 0.6256 - val_accuracy: 0.7860
Epoch 5/50
298/298 [==============================] - 24s 80ms/step - loss: 0.3086 - accuracy: 0.8925 - val_loss: 0.6596 - val_accuracy: 0.8000
Epoch 6/50
298/298 [==============================] - 24s 81ms/step - loss: 0.2481 - accuracy: 

2024/08/23 19:08:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run EfficientNetB1 at: http://127.0.0.1:5000/#/experiments/732603147262394224/runs/8d8a1b34114547719d452ef425ee58e6.
2024/08/23 19:08:34 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/732603147262394224.


======================================== EfficientNetB2 Model Training Started ========================================
31790344/31790344 [==============================] - 3s 0us/step
Epoch 1/50
298/298 [==============================] - 31s 87ms/step - loss: 1.8892 - accuracy: 0.7117 - val_loss: 0.6545 - val_accuracy: 0.7780
Epoch 2/50
298/298 [==============================] - 25s 82ms/step - loss: 0.5555 - accuracy: 0.8014 - val_loss: 0.6204 - val_accuracy: 0.7800
Epoch 3/50
298/298 [==============================] - 25s 83ms/step - loss: 0.4593 - accuracy: 0.8369 - val_loss: 0.5958 - val_accuracy: 0.7980
Epoch 4/50
298/298 [==============================] - 25s 84ms/step - loss: 0.3803 - accuracy: 0.8565 - val_loss: 0.5740 - val_accuracy: 0.7920
Epoch 5/50
298/298 [==============================] - 25s 82ms/step - loss: 0.3023 - accuracy: 0.8889 - val_loss: 0.5834 - val_accuracy: 0.7920
Epoch 6/50
298/298 [==============================] - 25s 83ms/step - loss: 0.2794 - accuracy: 

2024/08/23 19:29:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run EfficientNetB2 at: http://127.0.0.1:5000/#/experiments/732603147262394224/runs/ab05d1e7a86442f289c7ca1e6291376b.
2024/08/23 19:29:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/732603147262394224.


======================================== EfficientNetB3 Model Training Started ========================================
43941136/43941136 [==============================] - 4s 0us/step
Epoch 1/50
298/298 [==============================] - 36s 103ms/step - loss: 2.1282 - accuracy: 0.6919 - val_loss: 0.6685 - val_accuracy: 0.7460
Epoch 2/50
298/298 [==============================] - 29s 98ms/step - loss: 0.6391 - accuracy: 0.7711 - val_loss: 0.6465 - val_accuracy: 0.7740
Epoch 3/50
298/298 [==============================] - 30s 98ms/step - loss: 0.5297 - accuracy: 0.8102 - val_loss: 0.6581 - val_accuracy: 0.7540
Epoch 4/50
298/298 [==============================] - 30s 98ms/step - loss: 0.4580 - accuracy: 0.8366 - val_loss: 0.6193 - val_accuracy: 0.7940
Epoch 5/50
298/298 [==============================] - 30s 99ms/step - loss: 0.3883 - accuracy: 0.8545 - val_loss: 0.5914 - val_accuracy: 0.7840
Epoch 6/50
298/298 [==============================] - 29s 98ms/step - loss: 0.3503 - accuracy:

...

2024/08/24 09:02:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run ResNet50V2 at: http://127.0.0.1:5000/#/experiments/732603147262394224/runs/1f501b94ce1244e6858768de601dc180.
2024/08/24 09:02:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/732603147262394224.


======================================== VGG19 Model Training Started ========================================
80134624/80134624 [==============================] - 8s 0us/step
Epoch 1/50
298/298 [==============================] - 36s 119ms/step - loss: 2.3018 - accuracy: 0.6867 - val_loss: 0.7981 - val_accuracy: 0.7220
Epoch 2/50
298/298 [==============================] - 37s 124ms/step - loss: 0.5316 - accuracy: 0.8139 - val_loss: 0.7190 - val_accuracy: 0.7620
Epoch 3/50
298/298 [==============================] - 37s 125ms/step - loss: 0.3768 - accuracy: 0.8665 - val_loss: 0.8250 - val_accuracy: 0.7620
Epoch 4/50
298/298 [==============================] - 36s 121ms/step - loss: 0.2574 - accuracy: 0.9073 - val_loss: 0.8912 - val_accuracy: 0.7380
Epoch 5/50
298/298 [==============================] - 36s 118ms/step - loss: 0.1954 - accuracy: 0.9340 - val_loss: 1.1481 - val_accuracy: 0.7720
Epoch 6/50
298/298 [==============================] - 35s 118ms/step - loss: 0.1606 - accuracy: 0.9

2024/08/24 09:31:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run VGG19 at: http://127.0.0.1:5000/#/experiments/732603147262394224/runs/6d89338c48cc41b9932602b88c0798e1.
2024/08/24 09:31:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/732603147262394224.


======================================== Xception Model Training Started ========================================
83683744/83683744 [==============================] - 8s 0us/step
Epoch 1/50
298/298 [==============================] - 28s 88ms/step - loss: 139.0976 - accuracy: 0.6019 - val_loss: 30.9464 - val_accuracy: 0.4640
Epoch 2/50
298/298 [==============================] - 26s 87ms/step - loss: 10.0984 - accuracy: 0.6121 - val_loss: 1.1863 - val_accuracy: 0.6700
Epoch 3/50
298/298 [==============================] - 26s 87ms/step - loss: 1.3209 - accuracy: 0.6721 - val_loss: 1.0567 - val_accuracy: 0.6720
Epoch 4/50
298/298 [==============================] - 26s 87ms/step - loss: 1.1766 - accuracy: 0.6734 - val_loss: 1.0668 - val_accuracy: 0.6680
Epoch 5/50
298/298 [==============================] - 26s 86ms/step - loss: 1.1368 - accuracy: 0.6725 - val_loss: 1.0888 - val_accuracy: 0.6680
Epoch 6/50
298/298 [==============================] - 26s 87ms/step - loss: 1.0715 - accuracy: 0.

2024/08/24 09:54:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run Xception at: http://127.0.0.1:5000/#/experiments/732603147262394224/runs/9c3119c4c42d463196da4ee503c96042.
2024/08/24 09:54:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/732603147262394224.


## Meta_Data feature Extractro

In [63]:
meta_data = pd.read_csv("./Dataset/HAM10000_metadata.csv")
meta_data

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [6]:
# drop the useless columns from the meta data
del meta_data["lesion_id"]

In [7]:
# info of the meta data
meta_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   image_id      10015 non-null  object 
 1   dx            10015 non-null  object 
 2   dx_type       10015 non-null  object 
 3   age           9958 non-null   float64
 4   sex           10015 non-null  object 
 5   localization  10015 non-null  object 
dtypes: float64(1), object(5)
memory usage: 469.6+ KB


In [8]:
# fill the missing values of the age column
meta_data["age"] = meta_data["age"].fillna(method="ffill")

C:\Users\KARIMI\AppData\Local\Temp\ipykernel_25428\1046813865.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  meta_data['age'] = meta_data['age'].fillna(method='ffill')


In [9]:
# info of the meta data
meta_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   image_id      10015 non-null  object 
 1   dx            10015 non-null  object 
 2   dx_type       10015 non-null  object 
 3   age           10015 non-null  float64
 4   sex           10015 non-null  object 
 5   localization  10015 non-null  object 
dtypes: float64(1), object(5)
memory usage: 469.6+ KB


In [11]:
# Function to convert numerical age to text
def age_to_text(age):
    return num2words(age)


# Apply the conversion function to the age column
meta_data["age_text"] = meta_data["age"].apply(age_to_text)

# drop the age numerical column
del meta_data["age"]

In [12]:
meta_data

,image_id,dx,dx_type,sex,localization,age_text
0,ISIC_0027419,bkl,histo,male,scalp,eighty
1,ISIC_0025030,bkl,histo,male,scalp,eighty
2,ISIC_0026769,bkl,histo,male,scalp,eighty
3,ISIC_0025661,bkl,histo,male,scalp,eighty
4,ISIC_0031633,bkl,histo,male,ear,seventy-five
...,...,...,...,...,...,...
10010,ISIC_0033084,akiec,histo,male,abdomen,forty
10011,ISIC_0033550,akiec,histo,male,abdomen,forty
10012,ISIC_0033536,akiec,histo,male,abdomen,forty
10013,ISIC_0032854,akiec,histo,male,face,eighty


In [13]:
np.unique(meta_data["dx"]), np.unique(meta_data["dx_type"])

(array(['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'], dtype=object),
 array(['confocal', 'consensus', 'follow_up', 'histo'], dtype=object))

In [14]:
dx_full_form = {
    "akiec": "Actinic Keratoses and Intraepidermal Carcinoma",
    "bcc": "Basal Cell Carcinoma",
    "bkl": "Benign Keratosis-Like Lesions",
    "df": "Dermatofibroma",
    "mel": "Melanoma",
    "nv": "Melanocytic Nevi",
    "vasc": "Vascular Lesions",
}

dx_type_full_from = {
    "confocal": "confocal Microscopy",
    "consensus": "consensus",
    "follow_up": "follow-up",
    "histo": "histopathology",
}

meta_data["dx_type"] = meta_data["dx_type"].map(dx_type_full_from)
meta_data["dx_Full"] = meta_data["dx"].map(dx_full_form)


In [15]:
meta_data

,image_id,dx,dx_type,sex,localization,age_text,dx_Full
0,ISIC_0027419,bkl,histopathology,male,scalp,eighty,Benign Keratosis-Like Lesions
1,ISIC_0025030,bkl,histopathology,male,scalp,eighty,Benign Keratosis-Like Lesions
2,ISIC_0026769,bkl,histopathology,male,scalp,eighty,Benign Keratosis-Like Lesions
3,ISIC_0025661,bkl,histopathology,male,scalp,eighty,Benign Keratosis-Like Lesions
4,ISIC_0031633,bkl,histopathology,male,ear,seventy-five,Benign Keratosis-Like Lesions
...,...,...,...,...,...,...,...
10010,ISIC_0033084,akiec,histopathology,male,abdomen,forty,Actinic Keratoses and Intraepidermal Carcinoma
10011,ISIC_0033550,akiec,histopathology,male,abdomen,forty,Actinic Keratoses and Intraepidermal Carcinoma
10012,ISIC_0033536,akiec,histopathology,male,abdomen,forty,Actinic Keratoses and Intraepidermal Carcinoma
10013,ISIC_0032854,akiec,histopathology,male,face,eighty,Actinic Keratoses and Intraepidermal Carcinoma


In [16]:
# number of patients with unknow sex we will convert it into other
np.unique(meta_data["dx_type"])
print(np.unique(meta_data["sex"]))
# print("Unknown number of patients:",meta_data[meta_data['sex'] == 'unknown'].values.shape[0])
meta_data["sex"] = meta_data["sex"].replace("unknown", "other")
print(np.unique(meta_data["sex"]))

['female' 'male' 'unknown']
['female' 'male' 'other']


In [17]:
sorted_data = meta_data.sort_values(by="dx")
sorted_data

,image_id,dx,dx_type,sex,localization,age_text,dx_Full
9803,ISIC_0024575,akiec,histopathology,female,upper extremity,seventy,Actinic Keratoses and Intraepidermal Carcinoma
9789,ISIC_0026645,akiec,histopathology,male,back,sixty-five,Actinic Keratoses and Intraepidermal Carcinoma
9790,ISIC_0024710,akiec,histopathology,male,back,seventy-five,Actinic Keratoses and Intraepidermal Carcinoma
9791,ISIC_0029315,akiec,histopathology,male,lower extremity,seventy-five,Actinic Keratoses and Intraepidermal Carcinoma
9792,ISIC_0029811,akiec,histopathology,male,upper extremity,sixty-five,Actinic Keratoses and Intraepidermal Carcinoma
...,...,...,...,...,...,...,...
2412,ISIC_0031706,vasc,consensus,male,trunk,sixty,Vascular Lesions
2411,ISIC_0030283,vasc,consensus,male,trunk,fifty-five,Vascular Lesions
2410,ISIC_0031201,vasc,consensus,male,trunk,forty-five,Vascular Lesions
2417,ISIC_0032409,vasc,consensus,male,abdomen,eighty,Vascular Lesions


In [18]:
# # Convert the text into meaningful form
sorted_data["description"] = sorted_data.apply(
    lambda row: f'a {row["sex"]} patient, aged {row["age_text"]}, has been diagnosed with {row["dx_Full"]} the diagnosis was made through {row["dx_type"]} and it is located on the {row["localization"]}',
    axis=1,
)

In [19]:
# deleting all other columns except image_id
sorted_data = sorted_data.drop(
    columns=["dx_Full", "dx_type", "sex", "localization", "age_text"]
)
sorted_data

,image_id,dx,description
9803,ISIC_0024575,akiec,"a female patient, aged seventy, has been diagn..."
9789,ISIC_0026645,akiec,"a male patient, aged sixty-five, has been diag..."
9790,ISIC_0024710,akiec,"a male patient, aged seventy-five, has been di..."
9791,ISIC_0029315,akiec,"a male patient, aged seventy-five, has been di..."
9792,ISIC_0029811,akiec,"a male patient, aged sixty-five, has been diag..."
...,...,...,...
2412,ISIC_0031706,vasc,"a male patient, aged sixty, has been diagnosed..."
2411,ISIC_0030283,vasc,"a male patient, aged fifty-five, has been diag..."
2410,ISIC_0031201,vasc,"a male patient, aged forty-five, has been diag..."
2417,ISIC_0032409,vasc,"a male patient, aged eighty, has been diagnose..."


In [20]:
# saveing the file in json format
sorted_data.to_json("new_meta_data.json", orient="records", lines=True, indent=4)
sorted_data.to_csv("new_meta_data.csv", index=False)

### text vectorization

##### Text preprocessing
- Removing punctuations like . , ! $( ) * % @
- Removing URLs
- Removing Stop words
- Lower casing
- Tokenization

In [21]:
# library that contains punctuation
import string

print(f"list of Punctuation: {string.punctuation}")


# defining the function to remove punctua|tion
def remove_punctuation(text):
    punctuationfree = "".join([i for i in text if i not in string.punctuation])
    return punctuationfree


# storing the puntuation free text
sorted_data["CL_DS"] = sorted_data["description"].apply(lambda x: remove_punctuation(x))
sorted_data.head()

list of Punctuation: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


,image_id,dx,description,CL_DS
9803,ISIC_0024575,akiec,"a female patient, aged seventy, has been diagn...",a female patient aged seventy has been diagnos...
9789,ISIC_0026645,akiec,"a male patient, aged sixty-five, has been diag...",a male patient aged sixtyfive has been diagnos...
9790,ISIC_0024710,akiec,"a male patient, aged seventy-five, has been di...",a male patient aged seventyfive has been diagn...
9791,ISIC_0029315,akiec,"a male patient, aged seventy-five, has been di...",a male patient aged seventyfive has been diagn...
9792,ISIC_0029811,akiec,"a male patient, aged sixty-five, has been diag...",a male patient aged sixtyfive has been diagnos...


In [22]:
# lower casing: as our data is already in lower case but still we will go for it
sorted_data["CL_DS"] = sorted_data["CL_DS"].apply(lambda x: x.lower())

In [23]:
# defining function for tokenization
def tokenization(text):
    tokens = text.split()
    return tokens


# applying function to the column
sorted_data["msg_tokenied"] = sorted_data["CL_DS"].apply(lambda x: tokenization(x))

In [24]:
# stop words removal

# Stop words present in the library
stopwords = nltk.corpus.stopwords.words("english")

# defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    """Stopwords are removed in NLP to focus on the more meaningful and informative words in a text.
    This is often done to reduce noise, improve efficiency in processing,
    and highlight keywords that carry the essential meaning of the text."""

    output = [i for i in text if i not in stopwords]
    return output


# applying the function
sorted_data["no_stopwords"] = sorted_data["msg_tokenied"].apply(
    lambda x: remove_stopwords(x)
)

In [25]:
sorted_data

,image_id,dx,description,CL_DS,msg_tokenied,no_stopwords
9803,ISIC_0024575,akiec,"a female patient, aged seventy, has been diagn...",a female patient aged seventy has been diagnos...,"[a, female, patient, aged, seventy, has, been,...","[female, patient, aged, seventy, diagnosed, ac..."
9789,ISIC_0026645,akiec,"a male patient, aged sixty-five, has been diag...",a male patient aged sixtyfive has been diagnos...,"[a, male, patient, aged, sixtyfive, has, been,...","[male, patient, aged, sixtyfive, diagnosed, ac..."
9790,ISIC_0024710,akiec,"a male patient, aged seventy-five, has been di...",a male patient aged seventyfive has been diagn...,"[a, male, patient, aged, seventyfive, has, bee...","[male, patient, aged, seventyfive, diagnosed, ..."
9791,ISIC_0029315,akiec,"a male patient, aged seventy-five, has been di...",a male patient aged seventyfive has been diagn...,"[a, male, patient, aged, seventyfive, has, bee...","[male, patient, aged, seventyfive, diagnosed, ..."
9792,ISIC_0029811,akiec,"a male patient, aged sixty-five, has been diag...",a male patient aged sixtyfive has been diagnos...,"[a, male, patient, aged, sixtyfive, has, been,...","[male, patient, aged, sixtyfive, diagnosed, ac..."
...,...,...,...,...,...,...
2412,ISIC_0031706,vasc,"a male patient, aged sixty, has been diagnosed...",a male patient aged sixty has been diagnosed w...,"[a, male, patient, aged, sixty, has, been, dia...","[male, patient, aged, sixty, diagnosed, vascul..."
2411,ISIC_0030283,vasc,"a male patient, aged fifty-five, has been diag...",a male patient aged fiftyfive has been diagnos...,"[a, male, patient, aged, fiftyfive, has, been,...","[male, patient, aged, fiftyfive, diagnosed, va..."
2410,ISIC_0031201,vasc,"a male patient, aged forty-five, has been diag...",a male patient aged fortyfive has been diagnos...,"[a, male, patient, aged, fortyfive, has, been,...","[male, patient, aged, fortyfive, diagnosed, va..."
2417,ISIC_0032409,vasc,"a male patient, aged eighty, has been diagnose...",a male patient aged eighty has been diagnosed ...,"[a, male, patient, aged, eighty, has, been, di...","[male, patient, aged, eighty, diagnosed, vascu..."


In [26]:
# defining labels for text inputs
labels = sorted_data["dx"].values
text = [" ".join(val) for val in sorted_data["no_stopwords"].values]

labels_index = {val: i for i, val in enumerate(train_ds.class_names)}

In [27]:
labels = [labels_index[val] for val in labels]

In [28]:
# number of unique words in the dataset
len(set(" ".join(text).split()))

61

In [29]:
# text data splitting
train_set, test_set, train_labels, test_labels = train_test_split(
    text, labels, test_size=0.05, random_state=42
)

In [31]:
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(train_set)

train_sequences = tokenizer.texts_to_sequences(train_set)
test_sequences = tokenizer.texts_to_sequences(test_set)

word_index = tokenizer.word_index

print("Found %s unique tokens." % len(word_index))

Found 61 unique tokens.


In [32]:
train_set

['female patient aged seventyfive diagnosed basal cell carcinoma diagnosis made histopathology located lower extremity',
 'female patient aged sixty diagnosed melanoma diagnosis made histopathology located lower extremity',
 'male patient aged seventyfive diagnosed melanoma diagnosis made histopathology located back',
 'female patient aged fortyfive diagnosed melanocytic nevi diagnosis made histopathology located back',
 'female patient aged twentyfive diagnosed melanocytic nevi diagnosis made histopathology located unknown',
 'male patient aged seventyfive diagnosed melanocytic nevi diagnosis made histopathology located back',
 'male patient aged fiftyfive diagnosed melanocytic nevi diagnosis made followup located lower extremity',
 'female patient aged fifty diagnosed melanocytic nevi diagnosis made histopathology located lower extremity',
 'female patient aged seventy diagnosed benign keratosislike lesions diagnosis made histopathology located face',
 'female patient aged fifty diag

In [60]:
# converting this to sqequences to be fed into neural netwok
train_x = pad_sequences(train_sequences, maxlen=100)
test_x = pad_sequences(test_sequences, maxlen=100)

train_y = to_categorical(np.asarray(train_labels))
test_y = to_categorical(np.asarray(test_labels))

In [29]:
# Defining textual models
text_models = {}

text_model = Sequential()
text_model.add(Embedding(100, 64))
text_model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
text_model.add(Dense(7, activation="softmax"))

text_models["LSTM"] = text_model


text_model = Sequential()
text_model.add(Embedding(100, 64))
text_model.add(Conv1D(64, 5, activation="relu"))
text_model.add(MaxPooling1D(5))

text_model.add(Conv1D(64, 5, activation="relu"))
text_model.add(MaxPooling1D(5))

text_model.add(GlobalMaxPooling1D())

text_model.add(Dense(64, activation="relu"))
text_model.add(Dense(7, activation="softmax"))

text_models["CNN1D"] = text_model

In [35]:
# textual model training and tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Text Data Experiments")

for key in text_models.keys():
    text_models[key].compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    try:
        with mlflow.start_run(run_name=key, nested=True) as run:
            # log model parametes
            mlflow.log_param("Optimizer", "Adam")
            mlflow.log_param("Loss", "Categorical_Crossentropy")
            mlflow.log_param("Batch_size", 32)
            mlflow.log_param("Epochs", 50)

            # fit the model
            history = text_models[key].fit(
                train_x,
                train_y,
                batch_size=32,
                epochs=50,
                validation_data=(test_x, test_y),
            )

            # Log metrics
            for epoch in range(50):
                mlflow.log_metric(
                    "train_accuracy", history.history["accuracy"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "val_accuracy", history.history["val_accuracy"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "train_loss", history.history["loss"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "val_loss", history.history["val_loss"][epoch], step=epoch
                )

            # log the model
            # mlflow.keras.log_model(model,f"Keras-model-{key}")
    except:
        print("Someting Went Wrong")

Epoch 1/50
298/298 [==============================] - 174s 575ms/step - loss: 0.4408 - accuracy: 0.8709 - val_loss: 0.0330 - val_accuracy: 1.0000
Epoch 2/50
298/298 [==============================] - 170s 571ms/step - loss: 0.0117 - accuracy: 0.9997 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 3/50
298/298 [==============================] - 171s 575ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 4/50
298/298 [==============================] - 167s 561ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 6.5763e-04 - val_accuracy: 1.0000
Epoch 5/50
298/298 [==============================] - 181s 608ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 6/50
298/298 [==============================] - 178s 596ms/step - loss: 0.0010 - accuracy: 0.9998 - val_loss: 3.6139e-04 - val_accuracy: 1.0000
Epoch 7/50
298/298 [==============================] - 162s 542ms/step - loss: 2.9795e-04 - accuracy: 1.0000 - val_lo

2024/08/24 14:09:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run LSTM at: http://127.0.0.1:5000/#/experiments/768791717154720409/runs/bab7a6ee7c324feda0bef818010647e5.
2024/08/24 14:09:10 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/768791717154720409.


Epoch 1/50
298/298 [==============================] - 3s 5ms/step - loss: 0.2836 - accuracy: 0.9141 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 2/50
298/298 [==============================] - 1s 5ms/step - loss: 7.1685e-04 - accuracy: 1.0000 - val_loss: 2.6740e-04 - val_accuracy: 1.0000
Epoch 3/50
298/298 [==============================] - 1s 5ms/step - loss: 1.2990e-04 - accuracy: 1.0000 - val_loss: 9.5946e-05 - val_accuracy: 1.0000
Epoch 4/50
298/298 [==============================] - 1s 5ms/step - loss: 5.6325e-05 - accuracy: 1.0000 - val_loss: 5.0442e-05 - val_accuracy: 1.0000
Epoch 5/50
298/298 [==============================] - 1s 5ms/step - loss: 3.0905e-05 - accuracy: 1.0000 - val_loss: 3.0560e-05 - val_accuracy: 1.0000
Epoch 6/50
298/298 [==============================] - 1s 5ms/step - loss: 1.9341e-05 - accuracy: 1.0000 - val_loss: 1.9429e-05 - val_accuracy: 1.0000
Epoch 7/50
298/298 [==============================] - 1s 5ms/step - loss: 1.2869e-05 - accuracy: 1.0000 - va

2024/08/24 14:10:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run CNN1D at: http://127.0.0.1:5000/#/experiments/768791717154720409/runs/242aea257e064b72a449092a199be9a4.
2024/08/24 14:10:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/768791717154720409.


### Applying late fusion

In [30]:
# 1. creatiing a custom data generator

labels_index = {val: i for i, val in enumerate(train_ds.class_names)}

text = [" ".join(val) for val in sorted_data["no_stopwords"].values]
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(text)


def custom_dataGen(input, output, batch_size=32, image_size=(224, 224)):
    Labels = output.values
    Labels = [labels_index[val] for val in Labels]
    Labels = to_categorical(np.asarray(Labels))

    # merged_df = merged_df.sample(frac=1).reset_index(drop=True)
    num_samples = len(input["image_id"])
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_filenames = input["filename"][offset : offset + batch_size].values

            batch_labels = Labels[offset : offset + batch_size]
            # batch_labels = [labels_index[val] for val in batch_labels]
            # batch_labels = to_categorical(np.asarray(batch_labels))

            meta_data = input["no_stopwords"][offset : offset + batch_size].values
            meta_data = [" ".join(val) for val in meta_data]

            BatchMetaData = tokenizer.texts_to_sequences(meta_data)
            BatchMetaData = pad_sequences(BatchMetaData, maxlen=100)

            BatchImages = []
            for filename in batch_filenames:
                image = cv2.imread(filename)
                image = cv2.resize(image, image_size)

                BatchImages.append(image)

            BatchImages = np.array(BatchImages)
            BatchMetaData = np.array(BatchMetaData)
            yield (BatchImages, BatchMetaData), batch_labels

In [31]:
# sorted_data
extracted_ids = [
    filename.split("\\")[-1].split(".")[0]
    for filename in glob.glob("./HAM_10000_Dataset/*/*.jpg")
]

image_df = pd.DataFrame(
    {"image_id": extracted_ids, "filename": glob.glob("./HAM_10000_Dataset/*/*.jpg")}
)

merged_df = pd.merge(
    image_df, sorted_data[["image_id", "dx", "no_stopwords"]], on="image_id", how="left"
)

In [32]:
train_x, test_x, train_y, test_y = train_test_split(
    merged_df[["image_id", "filename", "no_stopwords"]],
    merged_df["dx"],
    test_size=0.05,
    random_state=123,
)

In [34]:
batch_size = 32

train_gen = custom_dataGen(train_x, train_y, batch_size=batch_size)
test_gen = custom_dataGen(test_x, test_y, batch_size=batch_size)

In [35]:
# modeling
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("LSTM Late Fusion Experiments")

for key in pretrained_models.keys():
    try:
        print("=" * 40, f"{key} Model Training Started", "=" * 40)

        PreImgModel = pretrained_models[key](
            include_top=False, input_shape=(224, 224, 3)
        )
        for layer in PreImgModel.layers:
            layer.trainable = False
        flat1 = keras.layers.Flatten()(PreImgModel.layers[-1].output)
        ImgFeatures = keras.layers.Dense(512, activation="relu")(flat1)
        image_output = keras.layers.Dense(7, activation="softmax")(flat1)

        ImgModel = keras.Model(inputs=PreImgModel.inputs, outputs=image_output)

        text_input = Input(
            shape=(100,), name="text_input"
        )  # Example input shape for text
        TextEmabedding = text_models["LSTM"].layers[0](text_input)
        TextFeatures = text_models["LSTM"].layers[1](TextEmabedding)
        text_output = text_models["LSTM"].layers[-1](TextFeatures)

        TextModel = keras.Model(inputs=text_input, outputs=text_output)

        combined_features = Concatenate()([TextFeatures, ImgFeatures])

        # Add some fully connected layers
        x = Dense(64, activation="relu")(combined_features)
        x = Dense(32, activation="relu")(x)
        final_output = Dense(7, activation="softmax")(
            x
        )

        # Define the combined model
        combined_model = Model(
            inputs=[PreImgModel.inputs, TextModel.inputs], outputs=final_output
        )

        # model compilation
        combined_model.compile(
            optimizer=keras.optimizers.Adam(),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )

        early_stopping = EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        )

        with mlflow.start_run(run_name=f"{key}-LSTM", nested=True) as run:
            # log model parametes
            mlflow.log_param("Optimizer", "Adam")
            mlflow.log_param("Loss", "Sparse_Categorical_Crossentropy")
            mlflow.log_param("Batch_size", 32)
            mlflow.log_param("Epochs", 20)

            # Define the number of steps per epoch and validation steps
            steps_per_epoch = len(train_x["image_id"]) // batch_size
            validation_steps = len(test_x["image_id"]) // batch_size

            # Fit the combined model using the generators
            history = combined_model.fit(
                train_gen,
                steps_per_epoch=steps_per_epoch,
                validation_data=test_gen,
                validation_steps=validation_steps,
                epochs=20,
                callbacks=[early_stopping],
            )

            # Log metrics
            for epoch in range(20):
                mlflow.log_metric(
                    "train_accuracy", history.history["accuracy"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "val_accuracy", history.history["val_accuracy"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "train_loss", history.history["loss"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "val_loss", history.history["val_loss"][epoch], step=epoch
                )

                # log the model
                # mlflow.keras.log_model(model,f"Keras-model-{key}")
    except Exception as e:
        print(e)

======================================== MobileNetV2 Model Training Started ========================================
Epoch 1/20
297/297 [==============================] - 150s 491ms/step - loss: 2.0265 - accuracy: 0.6394 - val_loss: 0.7893 - val_accuracy: 0.6979
Epoch 2/20
297/297 [==============================] - 145s 488ms/step - loss: 0.5552 - accuracy: 0.8130 - val_loss: 0.4144 - val_accuracy: 0.8333
Epoch 3/20
297/297 [==============================] - 145s 487ms/step - loss: 0.2309 - accuracy: 0.9248 - val_loss: 0.1927 - val_accuracy: 0.9208
Epoch 4/20
297/297 [==============================] - 144s 484ms/step - loss: 0.0871 - accuracy: 0.9746 - val_loss: 0.1077 - val_accuracy: 0.9708
Epoch 5/20
297/297 [==============================] - 143s 482ms/step - loss: 0.0339 - accuracy: 0.9909 - val_loss: 0.0500 - val_accuracy: 0.9833
Epoch 6/20
297/297 [==============================] - 145s 487ms/step - loss: 0.0117 - accuracy: 0.9983 - val_loss: 0.0317 - val_accuracy: 0.9854
Epoch 7

2024/08/26 15:37:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run MobileNetV2-LSTM at: http://127.0.0.1:5000/#/experiments/859317374638950073/runs/9dd17a0202c84f548471fe5199829770.
2024/08/26 15:37:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/859317374638950073.


======================================== MobileNetV3Large Model Training Started ========================================
Epoch 1/20
297/297 [==============================] - 150s 495ms/step - loss: 1.1216 - accuracy: 0.7234 - val_loss: 0.5941 - val_accuracy: 0.8125
Epoch 2/20
297/297 [==============================] - 145s 490ms/step - loss: 0.5762 - accuracy: 0.7965 - val_loss: 0.5468 - val_accuracy: 0.8146
Epoch 3/20
297/297 [==============================] - 141s 475ms/step - loss: 0.4168 - accuracy: 0.8477 - val_loss: 0.3419 - val_accuracy: 0.8771
Epoch 4/20
297/297 [==============================] - 138s 463ms/step - loss: 0.2589 - accuracy: 0.8974 - val_loss: 0.2875 - val_accuracy: 0.9021
Epoch 5/20
297/297 [==============================] - 138s 464ms/step - loss: 0.1334 - accuracy: 0.9491 - val_loss: 0.1344 - val_accuracy: 0.9583
Epoch 6/20
297/297 [==============================] - 140s 470ms/step - loss: 0.0560 - accuracy: 0.9802 - val_loss: 0.1125 - val_accuracy: 0.9625
Ep

2024/08/26 16:15:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run MobileNetV3Large-LSTM at: http://127.0.0.1:5000/#/experiments/859317374638950073/runs/91ab253255f74c8abe2f7b97c1d98b27.
2024/08/26 16:15:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/859317374638950073.


Someting Went Wrong
======================================== MobileNetV3Small Model Training Started ========================================
Epoch 1/20
297/297 [==============================] - 148s 487ms/step - loss: 0.6439 - accuracy: 0.8141 - val_loss: 0.1908 - val_accuracy: 0.9604
Epoch 2/20
297/297 [==============================] - 144s 485ms/step - loss: 0.0715 - accuracy: 0.9844 - val_loss: 0.0823 - val_accuracy: 0.9854
Epoch 3/20
297/297 [==============================] - 143s 481ms/step - loss: 0.0115 - accuracy: 0.9993 - val_loss: 0.0721 - val_accuracy: 0.9896
Epoch 4/20
297/297 [==============================] - 144s 486ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0545 - val_accuracy: 0.9875
Epoch 5/20
297/297 [==============================] - 142s 479ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0479 - val_accuracy: 0.9896
Epoch 6/20
297/297 [==============================] - 143s 482ms/step - loss: 9.1451e-04 - accuracy: 1.0000 - val_loss: 0.0468 -

...

2024/08/26 22:47:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run ResNet50V2-LSTM at: http://127.0.0.1:5000/#/experiments/859317374638950073/runs/b14161ba0ba546978d9248986d4ddfbc.
2024/08/26 22:47:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/859317374638950073.


======================================== VGG19 Model Training Started ========================================
Epoch 1/20
297/297 [==============================] - 143s 469ms/step - loss: 1.5533 - accuracy: 0.7788 - val_loss: 0.2816 - val_accuracy: 0.9062
Epoch 2/20
297/297 [==============================] - 141s 474ms/step - loss: 0.1120 - accuracy: 0.9641 - val_loss: 0.1168 - val_accuracy: 0.9604
Epoch 3/20
297/297 [==============================] - 142s 477ms/step - loss: 0.0317 - accuracy: 0.9938 - val_loss: 0.1247 - val_accuracy: 0.9646
Epoch 4/20
297/297 [==============================] - 139s 468ms/step - loss: 0.0192 - accuracy: 0.9963 - val_loss: 0.1506 - val_accuracy: 0.9583
Epoch 5/20
297/297 [==============================] - 139s 467ms/step - loss: 0.0049 - accuracy: 0.9996 - val_loss: 0.0893 - val_accuracy: 0.9646
Epoch 6/20
297/297 [==============================] - 140s 470ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0604 - val_accuracy: 0.9833
Epoch 7/20
29

2024/08/26 23:13:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run VGG19-LSTM at: http://127.0.0.1:5000/#/experiments/859317374638950073/runs/3923398e337c476db7e6b686aa11eb67.
2024/08/26 23:13:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/859317374638950073.


Someting Went Wrong
======================================== Xception Model Training Started ========================================
Epoch 1/20
297/297 [==============================] - 144s 475ms/step - loss: 62.0892 - accuracy: 0.5947 - val_loss: 31.3264 - val_accuracy: 0.5146
Epoch 2/20
297/297 [==============================] - 141s 475ms/step - loss: 12.8813 - accuracy: 0.6381 - val_loss: 16.7211 - val_accuracy: 0.5625
Epoch 3/20
297/297 [==============================] - 143s 481ms/step - loss: 5.8477 - accuracy: 0.6772 - val_loss: 4.8422 - val_accuracy: 0.6917
Epoch 4/20
297/297 [==============================] - 141s 475ms/step - loss: 4.3009 - accuracy: 0.6937 - val_loss: 2.3358 - val_accuracy: 0.6292
Epoch 5/20
297/297 [==============================] - 141s 475ms/step - loss: 1.6998 - accuracy: 0.7478 - val_loss: 1.7300 - val_accuracy: 0.7542
Epoch 6/20
297/297 [==============================] - 139s 468ms/step - loss: 1.3275 - accuracy: 0.7723 - val_loss: 1.1407 - val_acc

2024/08/27 00:00:28 INFO mlflow.tracking._tracking_service.client: 🏃 View run Xception-LSTM at: http://127.0.0.1:5000/#/experiments/859317374638950073/runs/aa151c68c003421591e399c0bbce7f94.
2024/08/27 00:00:28 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/859317374638950073.


In [38]:
# CNN1D model training
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("CNN1D Late Fusion Experiments")

for key in pretrained_models.keys():
    try:
        print("=" * 40, f"{key} Model Training Started", "=" * 40)

        PreImgModel = pretrained_models[key](
            include_top=False, input_shape=(224, 224, 3)
        )
        for layer in PreImgModel.layers:
            layer.trainable = False
        flat1 = keras.layers.Flatten()(PreImgModel.layers[-1].output)
        ImgFeatures = keras.layers.Dense(512, activation="relu")(flat1)
        image_output = keras.layers.Dense(7, activation="softmax")(flat1)

        ImgModel = keras.Model(inputs=PreImgModel.inputs, outputs=image_output)

        text_input = Input(
            shape=(100,), name="text_input"
        )  # Example input shape for text
        text_output = text_models["CNN1D"].layers[0](text_input)
        text_output = text_models["CNN1D"].layers[1](text_output)
        text_output = text_models["CNN1D"].layers[2](text_output)
        text_output = text_models["CNN1D"].layers[3](text_output)
        text_output = text_models["CNN1D"].layers[4](text_output)
        text_output = text_models["CNN1D"].layers[5](text_output)
        TextFeatures = text_models["CNN1D"].layers[6](text_output)
        text_output = text_models["CNN1D"].layers[7](TextFeatures)
        
        TextModel = keras.Model(inputs=text_input, outputs=text_output)

        combined_features = Concatenate()([TextFeatures, ImgFeatures])

        # Add some fully connected layers
        x = Dense(64, activation="relu")(combined_features)
        x = Dense(32, activation="relu")(x)
        final_output = Dense(7, activation="softmax")(
            x
        )  
        
        # Define the combined model
        combined_model = Model(
            inputs=[PreImgModel.inputs, TextModel.inputs], outputs=final_output
        )

        # model compilation
        combined_model.compile(
            optimizer=keras.optimizers.Adam(),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )

        early_stopping = EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        )

        with mlflow.start_run(run_name=f"{key}-CNN1D", nested=True) as run:
            # log model parametes
            mlflow.log_param("Optimizer", "Adam")
            mlflow.log_param("Loss", "Sparse_Categorical_Crossentropy")
            mlflow.log_param("Batch_size", 32)
            mlflow.log_param("Epochs", 20)

            # Define the number of steps per epoch and validation steps
            steps_per_epoch = len(train_x["image_id"]) // batch_size
            validation_steps = len(test_x["image_id"]) // batch_size

            # Fit the combined model using the generators
            history = combined_model.fit(
                train_gen,
                steps_per_epoch=steps_per_epoch,
                validation_data=test_gen,
                validation_steps=validation_steps,
                epochs=20,
                callbacks=[early_stopping],
            )

            # Log metrics
            for epoch in range(20):
                mlflow.log_metric(
                    "train_accuracy", history.history["accuracy"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "val_accuracy", history.history["val_accuracy"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "train_loss", history.history["loss"][epoch], step=epoch
                )
                mlflow.log_metric(
                    "val_loss", history.history["val_loss"][epoch], step=epoch
                )

            # log the model
            # mlflow.keras.log_model(model,f"Keras-model-{key}")
    except Exception as e:
        print(e)

======================================== VGG16 Model Training Started ========================================
Epoch 1/20
297/297 [==============================] - 44s 148ms/step - loss: 11.4186 - accuracy: 0.9839 - val_loss: 0.0074 - val_accuracy: 0.9979
Epoch 2/20
297/297 [==============================] - 43s 144ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0074 - val_accuracy: 0.9979
Epoch 3/20
297/297 [==============================] - 43s 145ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0074 - val_accuracy: 0.9979
Epoch 4/20
297/297 [==============================] - 43s 145ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0074 - val_accuracy: 0.9979
Epoch 5/20
297/297 [==============================] - 43s 146ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0074 - val_accuracy: 0.9979
Epoch 6/20
297/297 [==============================] - 43s 146ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0074 - val_accuracy: 0.997

2024/08/27 13:38:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run VGG16-LSTM at: http://127.0.0.1:5000/#/experiments/367792037536348237/runs/0e1ab974edd34aa49216860b4f724df7.
2024/08/27 13:38:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/367792037536348237.


Someting Went Wrong
======================================== ConvNeXtBase Model Training Started ========================================
Epoch 1/20
297/297 [==============================] - 60s 170ms/step - loss: 34.3074 - accuracy: 0.9738 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/20
297/297 [==============================] - 47s 158ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
297/297 [==============================] - 47s 158ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
297/297 [==============================] - 47s 159ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
297/297 [==============================] - 47s 159ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
297/297 [==============================] - 47s 160ms/step - loss: 0.0000e+00 - accuracy: 

2024/08/27 13:43:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run ConvNeXtBase-LSTM at: http://127.0.0.1:5000/#/experiments/367792037536348237/runs/78a46a3ace1842629a3d851f2a98902d.
2024/08/27 13:43:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/367792037536348237.


Someting Went Wrong
======================================== ConvNeXtLarge Model Training Started ========================================
Epoch 1/20
297/297 [==============================] - 85s 253ms/step - loss: 2.1283 - accuracy: 0.9921 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/20
297/297 [==============================] - 72s 241ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
297/297 [==============================] - 71s 240ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
297/297 [==============================] - 71s 240ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
297/297 [==============================] - 71s 240ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
297/297 [==============================] - 72s 241ms/step - loss: 0.0000e+00 - accuracy: 

...

Someting Went Wrong
======================================== InceptionV3 Model Training Started ========================================
Epoch 1/20
297/297 [==============================] - 48s 154ms/step - loss: 36.3016 - accuracy: 0.9803 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/20
297/297 [==============================] - 45s 150ms/step - loss: 1.4509 - accuracy: 0.9974 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
297/297 [==============================] - 45s 150ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
297/297 [==============================] - 45s 150ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
297/297 [==============================] - 44s 150ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
297/297 [==============================] - 45s 150ms/step - loss: 0.0000e+00 - accuracy: 1.000

2024/08/27 15:43:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run InceptionV3-LSTM at: http://127.0.0.1:5000/#/experiments/367792037536348237/runs/9aaadd5b577b44c29fbdc8b0c1b59f59.
2024/08/27 15:43:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/367792037536348237.


Someting Went Wrong
======================================== MobileNet Model Training Started ========================================
Epoch 1/20
297/297 [==============================] - 45s 150ms/step - loss: 34.1264 - accuracy: 0.9803 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/20
297/297 [==============================] - 44s 148ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
297/297 [==============================] - 44s 148ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
297/297 [==============================] - 44s 148ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
297/297 [==============================] - 44s 148ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
297/297 [==============================] - 44s 148ms/step - loss: 0.0000e+00 - accuracy: 1.0

2024/08/27 15:48:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run MobileNet-LSTM at: http://127.0.0.1:5000/#/experiments/367792037536348237/runs/4334cf0802c3498f9cc51adb4d5439d5.
2024/08/27 15:48:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/367792037536348237.


Someting Went Wrong
